In [ ]:
# ManyTX 全日期闭集 SNR循环实验 ResNet 150TX
from joblib import load
import pandas as pd
import numpy as np
import os
from data_utilities import *
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc
from tqdm.auto import tqdm
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import seaborn as sns
from datetime import datetime

dataset_name = 'ManyTx'
dataset_path='../ManyTx.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])

# ------------------------ 去掉不参与训练的 TX ------------------------
tx_list = compact_dataset['tx_list']
exclude_txs = ['12-1', '18-1', '19-4', '20-3', '7-12', '9-14']
tx_list = [tx for tx in tx_list if tx not in exclude_txs]

print(f"✅ 剩余用于训练的 TX 数量: {len(tx_list)}")
print(f"具体为：{tx_list}")

rx_list = compact_dataset['rx_list']
equalized = 0
capture_date_list = compact_dataset['capture_date_list']

print("闭集合训练，所有日期为：", capture_date_list)

n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)

X_train, y_train, X_test, y_test = preprocess_dataset_for_classification_random_split(
    compact_dataset=compact_dataset,
    tx_list=tx_list,
    rx_list=rx_list,
    test_ratio=0.25,
    max_sig=None,         # 每个信号最大截取数量，不截取就用 None
    equalized=0,          # 使用的数据类型
    use_phase_differential=False,  # 是否使用相位差分
    seed=42               # 随机种子，可复现
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test  shape:", X_test.shape)
print("y_test  shape:", y_test.shape)

# === 信号处理参数 ===
fs = 20e6
fc = 2.4e9
v = 120
Add_noise = True
Add_doppler = True

def compute_doppler_shift(v, fc):
    c = 3e8
    v = v / 3.6
    return (v / c) * fc

fd = compute_doppler_shift(v, fc)

def add_doppler_shift(signal, fd, fs):
    num_samples = signal.shape[-1]
    t = np.arange(num_samples) / fs
    doppler_phase = np.exp(1j * 2 * np.pi * fd * t)
    return signal * doppler_phase

def measure_snr(clean_signal, noisy_signal):
    signal_power = np.mean(np.abs(clean_signal) ** 2)
    noise = noisy_signal - clean_signal
    noise_power = np.mean(np.abs(noise) ** 2)
    if noise_power == 0:
        return float('inf')
    return 10 * np.log10(signal_power / noise_power)

def add_complex_awgn(signal, snr_db):
    signal_power = np.mean(np.abs(signal) ** 2)
    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear
    noise_std = np.sqrt(noise_power / 2)
    noise = np.random.normal(0, noise_std, signal.shape) + 1j*np.random.normal(0, noise_std, signal.shape)
    return signal + noise, noise

def preprocess_iq_data(data_real_imag, snr_db=None, fd=None, fs=None, add_noise=True, add_doppler=True, verify_snr=True):
    if add_noise and snr_db is None:
        raise ValueError("当add_noise=True时，必须提供snr_db参数")
    if add_doppler and (fd is None or fs is None):
        raise ValueError("当add_doppler=True时，必须提供fd和fs参数")
    data_complex = data_real_imag[...,0] + 1j*data_real_imag[...,1]
    processed = []
    snr_measured_list = []
    for i, sig in enumerate(data_complex):
         # 原始信号归一化
        sig = sig / (np.sqrt(np.mean(np.abs(sig)**2)) + 1e-12)
        current_sig = sig.copy()
        if add_doppler:
            current_sig = add_doppler_shift(current_sig, fd, fs)
        if add_noise:
            noisy_sig, _ = add_complex_awgn(current_sig, snr_db)
            current_sig = noisy_sig
            if verify_snr:
                measured_snr = measure_snr(sig, noisy_sig)
                snr_measured_list.append(measured_snr)
        processed.append(current_sig)
    processed = np.array(processed)
    processed_real_imag = np.stack([processed.real, processed.imag], axis=-1)
    return processed_real_imag, None


# === ResNet 1D 模型定义 ===
class BasicBlock1D(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, downsample=None, dropout=0.0):
        super().__init__()
        self.conv1 = nn.Conv1d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=dropout)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        return self.relu(out)

class ResNet18_1D(nn.Module):
    def __init__(self, num_classes=10, in_planes=64, dropout=0.0):
        super().__init__()
        self.in_planes = in_planes
        self.conv1 = nn.Conv1d(2, in_planes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(64, 2, stride=1, dropout=dropout)
        self.layer2 = self._make_layer(128, 2, stride=2, dropout=dropout)
        self.layer3 = self._make_layer(256, 2, stride=2, dropout=dropout)
        self.layer4 = self._make_layer(512, 2, stride=2, dropout=dropout)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)
    def _make_layer(self, planes, blocks, stride, dropout):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes)
            )
        layers = [BasicBlock1D(self.in_planes, planes, stride, downsample, dropout)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlock1D(self.in_planes, planes, dropout=dropout))
        return nn.Sequential(*layers)
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.squeeze(-1)
        return self.fc(x)

# === 训练超参数 ===
batch_size   = 64
num_epochs   = 200
learning_rate = 1e-4
weight_decay = 1e-3
in_planes    = 64
dropout      = 0.5
patience     = 5
n_splits     = 5
num_classes  = len(np.unique(y_train))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def compute_grad_norm(model):
    total_norm = 0.0
    for p in model.parameters():
        if p.grad is not None:
            total_norm += (p.grad.data.norm(2).item() ** 2)
    return total_norm ** 0.5

def moving_average(x, w=5):
    return np.convolve(x, np.ones(w), 'valid') / w

# === SNR 循环 ===
snr_list = list(range(20, -45, -5))  # 20,15,...,-40
all_results = {}

for SNR_dB in snr_list:
    print(f"\n===== 开始 SNR={SNR_dB} dB 实验 =====")

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    script_name = "wisig_time"
    folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}dB_fd{int(fd)}_classes_{num_classes}_ResNet18"
    save_folder = os.path.join(os.getcwd(), "training_results", folder_name)
    os.makedirs(save_folder, exist_ok=True)

    results_file = os.path.join(save_folder, "results.txt")
    with open(results_file, "w") as f:
        f.write(f"=== Experiment Summary ===\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"Total Classes: {num_classes}\n")
        f.write(f"SNR: {SNR_dB} dB\n")
        f.write(f"fd (Doppler shift): {fd} Hz\n")
        f.write(f"equalized: {equalized}\n")

    # === 数据处理 ===
    X_train_processed, _ = preprocess_iq_data(
        X_train, snr_db=SNR_dB, fd=fd, fs=fs, add_noise=Add_noise, add_doppler=Add_doppler, verify_snr=False
    )
    X_test_processed, _ = preprocess_iq_data(
        X_test, snr_db=SNR_dB, fd=fd, fs=fs, add_noise=Add_noise, add_doppler=Add_doppler, verify_snr=False
    )

    train_dataset = TensorDataset(torch.tensor(X_train_processed,dtype=torch.float32),
                                  torch.tensor(y_train,dtype=torch.long))
    test_dataset = TensorDataset(torch.tensor(X_test_processed,dtype=torch.float32),
                                 torch.tensor(y_test,dtype=torch.long))
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_results = []
    test_results = []
    avg_grad_norms_per_fold = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset)):
        print(f"\n=== Fold {fold+1}/{n_splits} ===")
        train_subset = Subset(train_dataset, train_idx)
        val_subset = Subset(train_dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, drop_last=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, drop_last=True)

        model = ResNet18_1D(num_classes=num_classes, in_planes=in_planes, dropout=dropout).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

        train_losses, val_losses = [], []
        train_accuracies, val_accuracies = [], []
        grad_norms = []
        best_val_loss = float('inf')
        patience_counter = 0

        for epoch in range(num_epochs):
            model.train()
            running_train_loss, correct_train, total_train = 0.0, 0, 0
            batch_grad_norms = []
            with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
                for inputs, labels in tepoch:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    grad_norm = compute_grad_norm(model)
                    batch_grad_norms.append(grad_norm)
                    optimizer.step()
                    running_train_loss += loss.item()
                    _, predicted = torch.max(outputs, 1)
                    total_train += labels.size(0)
                    correct_train += (predicted == labels).sum().item()
                    tepoch.set_postfix(loss=running_train_loss/len(train_loader),
                                       accuracy=100*correct_train/total_train,
                                       grad_norm=grad_norm)

            epoch_train_loss = running_train_loss/len(train_loader)
            train_losses.append(epoch_train_loss)
            train_accuracies.append(100*correct_train/total_train)
            avg_grad_norm = np.mean(batch_grad_norms)
            grad_norms.append(avg_grad_norm)

            # 验证
            model.eval()
            running_val_loss, correct_val, total_val = 0.0, 0, 0
            with torch.no_grad():
                for val_inputs, val_labels in val_loader:
                    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                    val_outputs = model(val_inputs)
                    val_loss = criterion(val_outputs, val_labels)
                    running_val_loss += val_loss.item()
                    _, val_predicted = torch.max(val_outputs, 1)
                    total_val += val_labels.size(0)
                    correct_val += (val_predicted == val_labels).sum().item()
            epoch_val_loss = running_val_loss / len(val_loader)
            val_losses.append(epoch_val_loss)
            val_accuracies.append(100*correct_val/total_val)

            with open(results_file,'a') as f:
                f.write(f"Fold{fold+1} Epoch{epoch+1} | TrainAcc={train_accuracies[-1]:.2f}% | ValAcc={val_accuracies[-1]:.2f}% | "
                        f"TrainLoss={train_losses[-1]:.4f} | ValLoss={val_losses[-1]:.4f} | AvgGrad={avg_grad_norm:.4f}\n")

            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                patience_counter = 0
            else:
                patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping")
                break
            scheduler.step()

        fold_results.append(max(val_accuracies))
        avg_grad_norms_per_fold.append(grad_norms)

        # 绘图
        plt.figure()
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Val Loss')
        plt.plot(moving_average(train_losses), label='Train Loss Smooth', linestyle='--')
        plt.plot(moving_average(val_losses), label='Val Loss Smooth', linestyle='--')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Fold {fold+1} Loss Curve')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_loss_curve.png"))
        plt.close()

        plt.figure()
        plt.plot(grad_norms, label='Grad Norm')
        plt.xlabel('Epoch')
        plt.ylabel('Grad Norm')
        plt.title(f'Fold {fold+1} Grad Norm')
        plt.grid(True)
        plt.legend()
        plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_grad_norm.png"))
        plt.close()

        # 测试集评估
        model.eval()
        test_preds, test_true = [], []
        with torch.no_grad():
            for test_inputs, test_labels in test_loader:
                test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
                test_outputs = model(test_inputs)
                _, predicted = torch.max(test_outputs, 1)
                test_preds.extend(predicted.cpu().numpy())
                test_true.extend(test_labels.cpu().numpy())
        test_preds = np.array(test_preds)
        test_true = np.array(test_true)
        test_accuracy = 100*np.sum(test_preds==test_true)/len(test_true)
        test_results.append(test_accuracy)

        print(f"Fold {fold+1} Test Accuracy: {test_accuracy:.2f}%")
        with open(results_file, "a") as f:
            f.write(f"Fold {fold+1} Test Acc: {test_accuracy:.2f}%\n")

        cm = confusion_matrix(test_true, test_preds)
        plt.figure(figsize=(10,8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Fold {fold+1} Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_confusion_matrix.png"))
        plt.close()

    avg_val = np.mean(fold_results)
    avg_test = np.mean(test_results)
    with open(results_file, "a") as f:
        f.write(f"\n=== SNR={SNR_dB} Summary ===\n")
        f.write(f"Avg Val Acc: {avg_val:.2f}%, Avg Test Acc: {avg_test:.2f}%\n")
    print(f"SNR={SNR_dB} 完成: Avg Val={avg_val:.2f}%, Avg Test={avg_test:.2f}%")
    all_results[SNR_dB] = save_folder

print("\n===== 所有 SNR 实验完成 =====")
for snr, folder in all_results.items():
    print(f"SNR={snr} 结果保存在: {folder}")
